In [15]:
import pandas as pd, numpy as np, tensorflow as tf

In [17]:
df = blogs_data.loadBlogs().sample(frac=1).reset_index(drop=True)
vocab, reverse_vocab = blogs_data.loadVocab()
train_len, test_len = np.floor(len(df)*0.8), np.floor(len(df)*0.2)
train, test = df.ix[:train_len-1], df.ix[train_len:train_len + test_len]
df = None
train.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


,post_id,gender,age_bracket,string,as_numbers,length
0,176768,1,2,"she is from austin , texas , and sounds very b...","[54, 14, 57, 3557, 1, 1883, 1, 6, 817, 130, 69...",15
1,45414,0,0,i 'll have to look for my reasons and doors .,"[3, 123, 23, 5, 186, 16, 13, 981, 6, 2208, 2]",11
2,1547,1,2,i decided to get serious about me and my future .,"[3, 305, 5, 59, 978, 47, 19, 6, 13, 611, 2]",11
3,147312,0,1,update ( <UNK> ) : the dentist <UNK> <UNK> <#> .,"[1043, 43, 0, 39, 52, 4, 4030, 0, 0, 12, 2]",11
4,173708,0,0,you really do n't have to forget someone you (...,"[15, 69, 31, 28, 23, 5, 627, 184, 15, 43, 98, ...",17


In [18]:
class SimpleDataIterator():
    def __init__(self, df):
        self.df = df
        self.size = len(self.df)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n-1 > self.size:
            self.epochs += 1
            self.shuffle()
        res = self.df.ix[self.cursor:self.cursor+n-1]
        self.cursor += n
        return res['as_numbers'], res['gender']*3 + res['age_bracket'], res['length']


In [19]:
data = SimpleDataIterator(train)
d = data.next_batch(3)
print('Input sequences\n', d[0], end='\n\n')
print('Target values\n', d[1], end='\n\n')
print('Sequence lengths\n', d[2])


Input sequences
 0          [27, 109, 37, 9, 1, 32, 111, 5, 0, 2269, 2]
1    [3, 8417, 24, 132, 208, 620, 315, 108, 0, 24, ...
2     [10, 1426, 40, 7, 919, 1227, 21, 16, 27, 166, 2]
Name: as_numbers, dtype: object

Target values
 0    4
1    3
2    1
dtype: int64

Sequence lengths
 0    11
1    17
2    11
Name: length, dtype: int64


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [20]:
class PaddedDataIterator(SimpleDataIterator):
    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            self.shuffle()
        res = self.df.ix[self.cursor:self.cursor+n-1]
        self.cursor += n

        # Pad sequences with 0s so they are all the same length
        maxlen = max(res['length'])
        x = np.zeros([n, maxlen], dtype=np.int32)
        for i, x_i in enumerate(x):
            x_i[:res['length'].values[i]] = res['as_numbers'].values[i]

        return x, res['gender']*3 + res['age_bracket'], res['length']

In [21]:
data = PaddedDataIterator(train)
d = data.next_batch(3)
print('Input sequences\n', d[0], end='\n\n')

Input sequences
 [[2046    1   15   42  979   19    7 6466  271    7  332   50    0    0
     0    0    0    0    0    0]
 [ 255    3  269   12    8   91  159    3   17   35  133    1    6  921
     3   17  318  521 1172    2]
 [  32  291  349    4  131  537   33   38   32   42   90    5  349  702
   728   33   51   14  288   50]]



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [23]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def build_graph(
    vocab_size = len(vocab),
    state_size = 64,
    batch_size = 256,
    num_classes = 6):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
    keep_prob = tf.placeholder(tf.float32,[])

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    cell = tf.nn.rnn_cell.GRUCell(state_size)
    init_state = tf.get_variable('init_state', [1, state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen,
                                                 initial_state=init_state)

    # Add dropout, as the model otherwise quickly overfits
    rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)

    """
    Obtain the last relevant output. The best approach in the future will be to use:

        last_rnn_output = tf.gather_nd(rnn_outputs, tf.pack([tf.range(batch_size), seqlen-1], axis=1))

    which is the Tensorflow equivalent of numpy's rnn_outputs[range(30), seqlen-1, :], but the
    gradient for this op has not been implemented as of this writing.

    The below solution works, but throws a UserWarning re: the gradient.
    """
    idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)

    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
        'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }

def train_graph(graph, batch_size = 256, num_epochs = 10, iterator = PaddedDataIterator):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        tr = iterator(train)
        te = iterator(test)

        step, accuracy = 0, 0
        tr_losses, te_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: 0.6}
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_
            
            print("train: current epoch:", current_epoch)
            print(accuracy)

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                te_epoch = te.epochs
                while te.epochs == te_epoch:
                    step += 1
                    batch = te.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_
                    
                    print("train: current epoch:", current_epoch)
                    print(accuracy)

                te_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- te:", te_losses[-1])

    return tr_losses, te_losses

In [24]:
g = build_graph()
tr_losses, te_losses = train_graph(g)

/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


train: current epoch: 0
0.203125
train: current epoch: 0
0.375
train: current epoch: 0
0.55078125
train: current epoch: 0
0.7265625
train: current epoch: 0
0.94921875
train: current epoch: 0
1.15234375
train: current epoch: 0
1.43359375
train: current epoch: 0
1.65625
train: current epoch: 0
1.85546875
train: current epoch: 0
2.078125
train: current epoch: 0
2.296875
train: current epoch: 0
2.49609375
train: current epoch: 0
2.71875
train: current epoch: 0
2.953125
train: current epoch: 0
3.1640625
train: current epoch: 0
3.41796875
train: current epoch: 0
3.65625
train: current epoch: 0
3.85546875
train: current epoch: 0
4.0390625
train: current epoch: 0
4.24609375
train: current epoch: 0
4.4609375
train: current epoch: 0
4.67578125
train: current epoch: 0
4.91796875
train: current epoch: 0
5.15625
train: current epoch: 0
5.3515625
train: current epoch: 0
5.55859375
train: current epoch: 0
5.80859375
train: current epoch: 0
6.0625
train: current epoch: 0
6.296875
train: current epoch:

train: current epoch: 0
52.65625
train: current epoch: 0
52.90625
train: current epoch: 0
53.10546875
train: current epoch: 0
53.3125
train: current epoch: 0
53.546875
train: current epoch: 0
53.76953125
train: current epoch: 0
54.015625
train: current epoch: 0
54.21875
train: current epoch: 0
54.46484375
train: current epoch: 0
54.6796875
train: current epoch: 0
54.9375
train: current epoch: 0
55.125
train: current epoch: 0
55.328125
train: current epoch: 0
55.5546875
train: current epoch: 0
55.8046875
train: current epoch: 0
56.01171875
train: current epoch: 0
56.21484375
train: current epoch: 0
56.375
train: current epoch: 0
56.61328125
train: current epoch: 0
56.8359375
train: current epoch: 0
57.0703125
train: current epoch: 0
57.26953125
train: current epoch: 0
57.45703125
train: current epoch: 0
57.65625
train: current epoch: 0
57.890625
train: current epoch: 0
58.1171875
train: current epoch: 0
58.3515625
train: current epoch: 0
58.5859375
train: current epoch: 0
58.83984375
tr

train: current epoch: 0
108.5625
train: current epoch: 0
108.8125
train: current epoch: 0
109.05859375
train: current epoch: 0
109.37109375
train: current epoch: 0
109.6328125
train: current epoch: 0
109.8984375
train: current epoch: 0
110.18359375
train: current epoch: 0
110.41796875
train: current epoch: 0
110.6640625
train: current epoch: 0
110.9140625
train: current epoch: 0
111.16015625
train: current epoch: 0
111.42578125
train: current epoch: 0
111.703125
train: current epoch: 0
111.984375
train: current epoch: 0
112.28515625
train: current epoch: 0
112.54296875
train: current epoch: 0
112.80078125
train: current epoch: 0
113.0859375
train: current epoch: 0
113.36328125
train: current epoch: 0
113.6328125
train: current epoch: 0
113.88671875
train: current epoch: 0
114.1875
train: current epoch: 0
114.42578125
train: current epoch: 0
114.6953125
train: current epoch: 0
114.9296875
train: current epoch: 0
115.1875
train: current epoch: 0
115.49609375
train: current epoch: 0
115.7

train: current epoch: 0
173.26171875
train: current epoch: 0
173.51171875
train: current epoch: 0
173.76953125
train: current epoch: 0
174.0625
train: current epoch: 0
174.3203125
train: current epoch: 0
174.625
train: current epoch: 0
174.94921875
train: current epoch: 0
175.27734375
train: current epoch: 0
175.5625
train: current epoch: 0
175.828125
train: current epoch: 0
176.09765625
train: current epoch: 0
176.34375
train: current epoch: 0
176.640625
train: current epoch: 0
176.9453125
train: current epoch: 0
177.2421875
train: current epoch: 0
177.54296875
train: current epoch: 0
177.86328125
train: current epoch: 0
178.17578125
train: current epoch: 0
178.51171875
train: current epoch: 0
178.77734375
train: current epoch: 0
179.0625
train: current epoch: 0
179.375
train: current epoch: 0
179.6796875
train: current epoch: 0
180.02734375
train: current epoch: 0
180.31640625
train: current epoch: 0
180.625
train: current epoch: 0
180.921875
train: current epoch: 0
181.21484375
trai

train: current epoch: 0
241.921875
train: current epoch: 0
242.25390625
train: current epoch: 0
242.54296875
train: current epoch: 0
242.8515625
train: current epoch: 0
243.20703125
train: current epoch: 0
243.55859375
train: current epoch: 0
243.8671875
train: current epoch: 0
244.20703125
train: current epoch: 0
244.51171875
train: current epoch: 0
244.79296875
train: current epoch: 0
245.046875
train: current epoch: 0
245.3125
train: current epoch: 0
245.57421875
train: current epoch: 0
245.86328125
train: current epoch: 0
246.1796875
train: current epoch: 0
246.55078125
train: current epoch: 0
246.84765625
train: current epoch: 0
247.17578125


KeyboardInterrupt: 